In [1]:
import polars as pl
import os
import sys
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
sys.path.append("..")
%load_ext autoreload
%autoreload 2

In [2]:
from synthetic_data.tasks.writing import WritingScoreAnnotate, GutenbergBacktranslationFromTxt
from synthetic_data.utils import ldictl
# df = pl.read_parquet("pile.parquet")

# input_df: pl.DataFrame = pl.read_parquet("../dataset_files/gutenberg_backtranslate.parquet")
task = GutenbergBacktranslationFromTxt()

# mock_batch = ldictl(input_df.head(4).to_dicts())
dataset = task.load_custom()

/Users/brianfitzgerald/Documents/GitHub/smolmodels/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
task_out = task.format_input_conversation(dataset[0:16])

2025-03-16 08:53:43.872 | INFO     | synthetic_data.tasks.writing:_get_paragraph_chunks:329 - Chunk length: 720
2025-03-16 08:53:44.173 | INFO     | synthetic_data.tasks.writing:_get_paragraph_chunks:329 - Chunk length: 733
2025-03-16 08:53:44.174 | INFO     | synthetic_data.tasks.writing:_get_paragraph_chunks:329 - Chunk length: 703
2025-03-16 08:53:44.174 | INFO     | synthetic_data.tasks.writing:_get_paragraph_chunks:329 - Chunk length: 655
2025-03-16 08:53:44.175 | INFO     | synthetic_data.tasks.writing:_get_paragraph_chunks:329 - Chunk length: 695
2025-03-16 08:53:44.175 | INFO     | synthetic_data.tasks.writing:_get_paragraph_chunks:329 - Chunk length: 741
2025-03-16 08:53:44.176 | INFO     | synthetic_data.tasks.writing:_get_paragraph_chunks:329 - Chunk length: 645
2025-03-16 08:53:44.176 | INFO     | synthetic_data.tasks.writing:_get_paragraph_chunks:329 - Chunk length: 608
2025-03-16 08:53:44.177 | INFO     | synthetic_data.tasks.writing:_get_paragraph_chunks:329 - Chunk leng

In [ ]:
print(task_out[0][0]['content'])

In [3]:
from dotenv import load_dotenv
load_dotenv("../.env")

True

In [7]:
from synthetic_data.generation import get_generation_wrapper


generation_wrapper = get_generation_wrapper("gemini-2.0-flash")
completions = await generation_wrapper.generate(
    task_out
)

2025-03-16 08:53:48.474 | INFO     | synthetic_data.generation:get_generation_wrapper:427 - Logging in to Hugging Face Hub
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [8]:
output_rows_batch = task.format_output_rows(completions)

In [14]:
print(output_rows_batch[0]['instruction'])

Write a paragraph introducing several characters from an urban fantasy setting. The style should be concise and descriptive, similar to a character sheet or dossier. Each character entry should include their name, role/affiliation, gender, age (approximate), physical appearance, and a key skill or personality trait. Use short, declarative sentences and a straightforward, almost clinical tone.

Present five characters. The first should be a powerful leader of shapeshifters, giving details about their animal form and fighting style. The second should be another shapeshifter, emphasizing their security role, paranoia, and a past connection with a previous mentioned character that was not romantic. The third should be a smaller-statured character, contrasting their appearance with their legal expertise and protectiveness over a main character. The fourth character should be female, highlighting her unique heritage as beastkin (a hybrid shapeshifter), detailing her past trauma, incredible s

In [ ]:
import tiktoken
from synthetic_data.gutenberg_parser import super_cleaner
from synthetic_data.tasks.writing import find_valid_paragraph_chunks
from IPython.display import display, Markdown

tiktoken_encoder = tiktoken.get_encoding("o200k_base")

df_sampled = df.sample(100)
for row in df.iter_rows(named=True):
    text = row["text"]
    paragraphs = super_cleaner(text)

    # find chunks of consecutive paragraphs with dialogue and at least 3 sentences
    valid_chunks = find_valid_paragraph_chunks(paragraphs, tiktoken_encoder)
    display(Markdown(f"### {row['title']}"))
    for chunk in valid_chunks:
        display(Markdown(chunk[0]))

